<a href="https://colab.research.google.com/github/nvinogradskaya/DL_HW4_RNN/blob/main/People_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 12.3 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import pandas as pd
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from tqdm import tqdm
import shutil
from pathlib import Path
from ultralytics import YOLO
import numpy as np

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# датасет для загрузки данных
def prepare_subset(train_images_dir, train_annotations_dir, subset_images_dir, subset_annotations_dir, num_images=1000):
    os.makedirs(subset_images_dir, exist_ok=True)
    os.makedirs(subset_annotations_dir, exist_ok=True)

    image_files = sorted(Path(train_images_dir).glob("*.jpg"))[:num_images]

    for image_file in image_files:
        shutil.copy(image_file, Path(subset_images_dir) / image_file.name)
        annotation_file = Path(train_annotations_dir) / f"{image_file.stem}.txt"
        if annotation_file.exists():
            shutil.copy(annotation_file, Path(subset_annotations_dir) / annotation_file.name)

    print(f"Prepared subset with {num_images} images.")

# в формат YOLO
def convert_annotations_to_yolo(annotation_dir, images_dir, yolo_labels_dir):
    os.makedirs(yolo_labels_dir, exist_ok=True)

    for annotation_file in Path(annotation_dir).glob("*.txt"):
        image_file = Path(images_dir) / f"{annotation_file.stem}.jpg"
        if not image_file.exists():
            continue
        image = cv2.imread(str(image_file))
        height, width, _ = image.shape

        yolo_annotations = []
        with open(annotation_file, "r") as f:
            for line in f:
                coords = list(map(int, line.strip().split(",")))
                if len(coords) == 4:
                    x_min, y_min, x_max, y_max = coords
                    x_center = (x_min + x_max) / 2 / width
                    y_center = (y_min + y_max) / 2 / height
                    box_width = (x_max - x_min) / width
                    box_height = (y_max - y_min) / height
                    yolo_annotations.append(f"0 {x_center} {y_center} {box_width} {box_height}")
        with open(Path(yolo_labels_dir) / annotation_file.name, "w") as f:
            f.write("\n".join(yolo_annotations))

    print(f"Converted annotations to YOLO format in {yolo_labels_dir}.")

In [ ]:
def collate_fn(batch):
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    return images, targets

In [ ]:
#  подготовка данных
def prepare_data(image_dir, annotation_dir):
    from pathlib import Path
    image_dir = Path(image_dir)
    annotation_dir = Path(annotation_dir)

    image_files = list(image_dir.glob("*.jpg"))
    dataset = PeopleDataset(image_files, annotation_dir, transform=None)
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

    return dataloader

In [ ]:
# обучение модели
def train_yolo_model(data_config_path, model_path, epochs=5):
    model = YOLO("yolov8n.pt")
    model.train(data=data_config_path, epochs=epochs, imgsz=640)
    return model

In [ ]:
# предсказание
import csv
from pathlib import Path

def predict_and_save_yolo(model, test_images_dir, output_csv_path):
    from ultralytics import YOLO

    test_images_dir = Path(test_images_dir)
    output_csv_path = Path(output_csv_path)

    results = model.predict(source=str(test_images_dir), save=False, save_txt=False)
    with open(output_csv_path, mode="w", newline="") as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["ID", "bboxes"])
        for result in results:
            image_id = Path(result.path).stem
            bboxes = []

            for box in result.boxes.xyxy:
                x_min, y_min, x_max, y_max = map(int, box.tolist())
                bboxes.append((x_min, y_min, x_max, y_max))
            csvwriter.writerow([image_id, str(bboxes)])

    print(f"Итоговый файл сохранён: {output_csv_path}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
if __name__ == "__main__":
    train_images = "/content/drive/MyDrive/Colab Notebooks/train/images"
    train_annotations = "/content/drive/MyDrive/Colab Notebooks/train/annotations"
    test_images = "/content/drive/MyDrive/Colab Notebooks/test/images"
    output_dir = "/content/drive/MyDrive/Colab Notebooks/output"

    subset_images = "/content/drive/MyDrive/Colab Notebooks/train_subset/images"
    subset_annotations = "/content/drive/MyDrive/Colab Notebooks/train_subset/annotations"
    yolo_labels = "/content/drive/MyDrive/Colab Notebooks/train_subset/yolo_labels"
    yolo_data_config = "/content/drive/MyDrive/Colab Notebooks/yolo_data.yaml"

    prepare_subset(train_images, train_annotations, subset_images, subset_annotations, num_images=1000)

    convert_annotations_to_yolo(subset_annotations, subset_images, yolo_labels)
    with open(yolo_data_config, "w") as f:
        f.write(f"""
        train: {subset_images}
        val: {subset_images}
        nc: 1
        names: ['person']
        """)
    model = train_yolo_model(yolo_data_config, model_path="yolov8n.pt", epochs=5)

    predict_and_save_yolo(model, test_images, output_dir)

    print("ready")

Prepared subset with 1000 images.
Converted annotations to YOLO format in /content/drive/MyDrive/Colab Notebooks/train_subset/yolo_labels.


100%|██████████| 6.25M/6.25M [00:00<00:00, 51.2MB/s]


Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/Colab Notebooks/yolo_data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

100%|██████████| 755k/755k [00:00<00:00, 10.0MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...


5 epochs completed in 1.732 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [04:23<00:00,  8.25s/it]

                   all       1000          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


Speed: 6.6ms preprocess, 243.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/train


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1434 /content/drive/MyDrive/Colab Notebooks/test/images/100.jpg: 480x640 (no detections), 190.3ms
image 2/1434 /content/drive/MyDrive/Colab Notebooks/test/images/1000.jpg: 640x512 (no detections), 288.7ms
image 3/1434 /content/drive/MyDrive/Colab Notebooks/test/images/1006.jpg: 640x640 (no detections), 372.2ms
image 4/1434 /con

AttributeError: 'str' object has no attribute 'name'

In [ ]:
test_images = "/content/drive/MyDrive/Colab Notebooks/test/images"
output_csv = "/content/drive/MyDrive/Colab Notebooks/submission.csv"

predict_and_save_yolo(model, test_images, output_csv)




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1434 /content/drive/MyDrive/Colab Notebooks/test/images/100.jpg: 480x640 (no detections), 258.1ms
image 2/1434 /content/drive/MyDrive/Colab Notebooks/test/images/1000.jpg: 640x512 (no detections), 281.1ms
image 3/1434 /content/drive/MyDrive/Colab Notebooks/test/images/1006.jpg: 640x640 (no detections), 343.2ms
image 4/1434 /content/drive/MyDrive/Colab Notebooks/test/images/1007.jpg: 640x512 (no detections), 276.4ms
image 5/1434 /content/drive/M

In [ ]:
output_dir = "/content/drive/MyDrive/Colab Notebooks/output"